In [ ]:
from keras.applications import nasnet
from keras.optimizers import SGD, Adam, Nadam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau,EarlyStopping,CSVLogger
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

import os
from tqdm import tqdm
import time
import shutil
import h5py
from PIL import Image

## Training parameters

In [ ]:
input_shape = (256, 256, 3)

label_names = {0: 'farm', 1: 'houses', 2: 'others', 3: 'blackfarms'}
label_names_inv = {v: k for k,v in label_names.items()}
num_classes = len(label_names)

batch_size = 16
nb_epoch = 200

np.random.seed(42)

In [ ]:
def load_dataset(dataset_path):
    with h5py.File(dataset_path,'r') as f:
        dx = f['data_x'][:]
        dy = f['data_y'][:]  
    return dx, to_categorical(dy, num_classes)

def batch_generator(data_X, data_y, batch_size):
    indexes = np.array(range(len(data_y)))
    n = len(indexes)
    while True:
        batch_start = 0
        batch_end = batch_size
        np.random.shuffle(indexes)
        while batch_start < n:
            index = []
            batch_y = []
            y = []
            index = indexes[batch_start:batch_end]
            batch_x = np.array([data_X[i] for i in index])
            batch_y = np.array([data_y[i] for i in index])
            yield batch_x, batch_y
            batch_start += batch_size   
            batch_end += batch_size

In [ ]:
model_name = 'nasnet_large_2d_nadam'
dataset_path = 'dataset_2d_new.h5'
if not os.path.exists(model_name):
    os.mkdir(model_name)

## Load and split data

In [ ]:
X, y = load_dataset(dataset_path)
print ('Size of dataset:', len(y))
print(X.shape)

In [ ]:
X_train, y_train = X[1800:], y[1800:]
X_val, y_val = X[600:1200], y[600:1200]
X_test, y_test = X[:600], y[:600]
print('Training: {}\tValidation: {}\tTesting: {}'.format(X_train.shape[0], X_val.shape[0], X_test.shape[0]))

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
# print('Training: {}\tValidation: {}\tTesting: {}'.format(X_train.shape[0], X_val.shape[0], X_test.shape[0]))

In [ ]:
assert not np.any(np.isnan(X_train))
assert not np.any(np.isnan(y_train))
assert not np.any(np.isnan(X_val))
assert not np.any(np.isnan(y_val))
assert not np.any(np.isnan(X_test))
assert not np.any(np.isnan(y_test))

In [ ]:
train_gen = batch_generator(X_train, y_train, batch_size)
val_gen = batch_generator(X_val, y_val, batch_size)
test_gen = batch_generator(X_test, y_test, batch_size)

## Model and training

In [ ]:
# model = inception_v3.InceptionV3(include_top=True, weights=None, input_shape=input_shape, classes=num_classes)
model = nasnet.NASNetLarge(include_top=True, weights=None, input_shape=input_shape, classes=num_classes)
#model.load_weights(model_name+'/'+model_name+'.h5')

model.summary()
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), decay=0.0)
nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.004)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint(model_name+'/'+model_name+'.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')
tensorboard = TensorBoard(log_dir=model_name, batch_size=batch_size, write_graph=True, write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=2, mode='auto')
cvslogger = CSVLogger(model_name+'/'+model_name+'.csv', separator=',', append=True)

callbacks = [checkpoint, reduce_lr, cvslogger, tensorboard, earlystop]
train_steps = int(len(y_train)//batch_size)
val_steps = int(len(y_test)//batch_size)
history = model.fit_generator(train_gen, train_steps, epochs=nb_epoch, verbose=1, 
                    max_queue_size=2, validation_data=val_gen, 
                    validation_steps=val_steps, shuffle=True,
                    workers=1, use_multiprocessing= False, 
                    initial_epoch=0, callbacks=callbacks)

In [ ]:
model.summary()

In [ ]:
vx, vy = val_gen.next()
print (vx.shape, vy.shape)
image_input = vx[, :,:,:]
print (image_input.shape)

In [ ]:
#@### load best weights and make sub model
from keras.models import Model, load_model
model.load_weights (model_name+'/'+model_name+'.h5')
sub_model = Model (inputs = model.input, outputs = model.get_layer('activation_7').output)
sub_model.summary()

### compute activation of selected layer
sub_output = sub_model.predict (vx)
print (sub_output.shape)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages

feature = sub_output#[1,:,:,:]#model.predict(batch_imagesX)  # feature is activation of intermediate layer

pdf = PdfPages( 'test.pdf' )
gs = gridspec.GridSpec(7, 6, top=1., bottom=0., right=1., left=0., hspace=0.,
        wspace=0.)
i =0
for g in gs:
    ax = plt.subplot(g)
    ax.imshow(feature[2][:,:,i])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('auto')
    i+=1
    
pdf.savefig()
pdf.close()



## Results and evaluation

In [ ]:
model.metrics_names

In [ ]:
model.evaluate_generator(test_gen, steps=len(X_test)//batch_size)

In [ ]:
model.evaluate_generator(test_gen, steps=len(X_test)//batch_size)

In [ ]:
preds = model.predict(X_test)
preds_bin = (preds == preds.max(axis=1, keepdims=True)).astype(int)
preds.shape

In [ ]:
preds[0]

In [ ]:
preds[0]

In [ ]:
pres_classes = []
for p in preds.argmax(axis=1):
    pres_classes.append(label_names[p])
    

In [ ]:
truth_classes = []
for p in y_test.argmax(axis=1):
    truth_classes.append(label_names[p])

In [ ]:
import pandas as pd
df = pd.read_csv('comparison3.csv')
print(df.shape)
df.head()

In [ ]:
df['2d_pred'] = pres_classes
df.head()

In [ ]:
img = Image.fromarray(X_test[2], 'RGB')
img.save('my.png')
img.show()

In [ ]:
def transform(val):
    if val.split('_') == ['houses','to','houses']:
        return 'others'
    if val.split('_')[2] == 'houses':
        return 'construction'
    if val == 'blackfarms_to_farm':
        return 'cultivation'
    if val == 'farm_to_blackfarms':
        return 'uncultivation'
    return 'others'

In [ ]:
from itertools import product

cc = ['houses', 'farm', 'blackfarms', 'others']
for x in product(cc,cc):
    print(x)

In [ ]:
labels = []

for index in range(0, len(pres_classes), 3):
    pred11, pred13, pred17 = pres_classes[index:index+3]
    
    trans_11_13 = transform(pred11 + '_to_' + pred13)
    trans_13_17 = transform(pred13 + '_to_' + pred17)
    
    if trans_13_17 != 'others':
        final_label = trans_13_17
    else:
        final_label = trans_11_13
    
    labels.append(final_label)


In [ ]:
index

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('comparison3.csv')
df.head()

In [ ]:
df['2d_pred'] = labels

In [ ]:
df.head()

In [ ]:
df.shape()

In [ ]:
cols = df.columns.tolist()

In [ ]:
cols = cols[-1:] + cols[:-1]

In [ ]:
cols

In [ ]:
df = df[cols]
df.to_csv('comparison5.csv', index=False, header=True)
df.head()

In [ ]:
df.head()

### Compare accuracy

In [ ]:
(df['3d_pred'] == df['truth']).sum() / float(len(df))

In [ ]:
(df['2d_pred'] == df['truth']).sum() / float(len(df))

In [ ]:
(df['2d_inceptionv3'] == df['truth']).sum() / float(len(df))

In [ ]:
(df['2d_inception_resnetv2'] == df['truth']).sum() / float(len(df))

In [ ]:
np.sum(np.all(preds_bin == y_test.astype(int), axis=1)) / float(len(y_test))

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [ ]:
class_labels = {0: 'others', 1: 'construction', 2: 'cultivation', 3: 'uncultivation'}

In [ ]:
label_index = 0
for imgnum in tqdm(range(0,len(preds),3)):
#     imgnum = 112

    fig = plt.figure(figsize=(15, 20))
    plt.subplot(131)
    plt.imshow(X_test[imgnum][:,:,::-1])
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(X_test[imgnum+1][:,:,::-1])
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(X_test[imgnum+2][:,:,::-1])
    plt.axis('off')

    pred_label = labels[label_index]
#     truth = class_labels[y_test[imgnum].astype(int)[0]]

    preds_msg = 'pred: {}  /\  truth: X'.format(pred_label)

    fig.text(0.1, 0.6, preds_msg, fontsize=12)

    plt.savefig('final_results_2d/{}.png'.format(imgnum), bbox_inches='tight')
    plt.close()
    
    label_index += 1